In [ ]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import optuna

In [ ]:
# Пример данных о локациях
data = {
    'latitude': [40.7128, 40.7306, 40.7484, 40.7580, 40.7688, 40.7788, 40.7888, 40.7988],
    'longitude': [-74.0060, -73.9352, -73.9857, -73.9857, -73.9681, -73.9581, -73.9481, -73.9381],
    'idealness': [1, 1, 0, 0, 0, 0, 0, 0]  # Примерные оценки идеальности (1 - идеальная, 0 - неидеальная)
}

df = pd.DataFrame(data)

# Создаем трансформер для проекции Меркатора
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Преобразуем географические координаты в плоские
df['x'], df['y'] = transformer.transform(df['longitude'].values, df['latitude'].values)

# Функция для расчета манхэттенского расстояния
def manhattan_distance(x1, y1, x2, y2):
    return abs(x2 - x1) + abs(y2 - y1)

# Расчет матрицы манхэттенских расстояний
def compute_manhattan_distances(df):
    n = len(df)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = manhattan_distance(df.loc[i, 'x'], df.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])
    return distances

# Вычисляем матрицу манхэттенских расстояний
distances_matrix = compute_manhattan_distances(df)

# Добавляем расстояния до всех локаций в качестве признаков
distances_df = pd.DataFrame(distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
df = pd.concat([df, distances_df], axis=1)

# Разделяем данные на обучающую и тестовую выборки
X = df.drop(columns=['latitude', 'longitude', 'idealness'])
y = df['idealness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функция для оптимизации гиперпараметров с использованием Optuna
def objective(trial, model_type):
    if model_type == 'random_forest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'random_state': 42
        }
        model = RandomForestClassifier(**params)
    elif model_type == 'catboost':
        params = {
            'iterations': trial.suggest_int('iterations', 50, 200),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 10),
            'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 10),
            'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 10),
            'od_type': 'Iter',
            'od_wait': 10,
            'verbose': 0,
            'random_state': 42
        }
        model = CatBoostClassifier(**params)
    elif model_type == 'xgboost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10),
            'random_state': 42
        }
        model = XGBClassifier(**params)
    elif model_type == 'lightgbm':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10),
            'random_state': 42
        }
        model = LGBMClassifier(**params)
    else:
        raise ValueError("Unsupported model type")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    return f1

# Оптимизация гиперпараметров для RandomForest
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(lambda trial: objective(trial, 'random_forest'), n_trials=50)
best_params_rf = study_rf.best_params
print(f"Лучшие параметры для RandomForest: {best_params_rf}")

# Оптимизация гиперпараметров для CatBoost
study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(lambda trial: objective(trial, 'catboost'), n_trials=50)
best_params_catboost = study_catboost.best_params
print(f"Лучшие параметры для CatBoost: {best_params_catboost}")

# Оптимизация гиперпараметров для XGBoost
study_xgboost = optuna.create_study(direction='maximize')
study_xgboost.optimize(lambda trial: objective(trial, 'xgboost'), n_trials=50)
best_params_xgboost = study_xgboost.best_params
print(f"Лучшие параметры для XGBoost: {best_params_xgboost}")

# Оптимизация гиперпараметров для LightGBM
study_lightgbm = optuna.create_study(direction='maximize')
study_lightgbm.optimize(lambda trial: objective(trial, 'lightgbm'), n_trials=50)
best_params_lightgbm = study_lightgbm.best_params
print(f"Лучшие параметры для LightGBM: {best_params_lightgbm}")

# Обучение моделей с лучшими параметрами
best_model_rf = RandomForestClassifier(**best_params_rf)
best_model_rf.fit(X_train, y_train)

best_model_catboost = CatBoostClassifier(**best_params_catboost)
best_model_catboost.fit(X_train, y_train)

best_model_xgboost = XGBClassifier(**best_params_xgboost)
best_model_xgboost.fit(X_train, y_train)

best_model_lightgbm = LGBMClassifier(**best_params_lightgbm)
best_model_lightgbm.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred_rf = best_model_rf.predict(X_test)
y_pred_catboost = best_model_catboost.predict(X_test)
y_pred_xgboost = best_model_xgboost.predict(X_test)
y_pred_lightgbm = best_model_lightgbm.predict(X_test)

# Оцениваем качество моделей
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
print(f"Accuracy для RandomForest с оптимизированными параметрами: {accuracy_rf}")
print(f"F1-score для RandomForest с оптимизированными параметрами: {f1_rf}")

accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
f1_catboost = f1_score(y_test, y_pred_catboost)
print(f"Accuracy для CatBoost с оптимизированными параметрами: {accuracy_catboost}")
print(f"F1-score для CatBoost с оптимизированными параметрами: {f1_catboost}")

accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
print(f"Accuracy для XGBoost с оптимизированными параметрами: {accuracy_xgboost}")
print(f"F1-score для XGBoost с оптимизированными параметрами: {f1_xgboost}")

accuracy_lightgbm = accuracy_score(y_test, y_pred_lightgbm)
f1_lightgbm = f1_score(y_test, y_pred_lightgbm)
print(f"Accuracy для LightGBM с оптимизированными параметрами: {accuracy_lightgbm}")
print(f"F1-score для LightGBM с оптимизированными параметрами: {f1_lightgbm}")

# Кросс-валидация
cv_scores_rf = cross_val_score(best_model_rf, X, y, cv=5, scoring='f1')
f1_cv_rf = cv_scores_rf.mean()
print(f"F1-score после кросс-валидации для RandomForest: {f1_cv_rf}")

cv_scores_catboost = cross_val_score(best_model_catboost, X, y, cv=5, scoring='f1')
f1_cv_catboost = cv_scores_catboost.mean()
print(f"F1-score после кросс-валидации для CatBoost: {f1_cv_catboost}")

cv_scores_xgboost = cross_val_score(best_model_xgboost, X, y, cv=5, scoring='f1')
f1_cv_xgboost = cv_scores_xgboost.mean()
print(f"F1-score после кросс-валидации для XGBoost: {f1_cv_xgboost}")

cv_scores_lightgbm = cross_val_score(best_model_lightgbm, X, y, cv=5, scoring='f1')
f1_cv_lightgbm = cv_scores_lightgbm.mean()
print(f"F1-score после кросс-валидации для LightGBM: {f1_cv_lightgbm}")

# Предсказываем идеальность для новых локаций
new_locations = pd.DataFrame({
    'latitude': [40.7530, 40.7688],
    'longitude': [-73.9824, -73.9681]
})

new_locations['x'], new_locations['y'] = transformer.transform(new_locations['longitude'].values, new_locations['latitude'].values)

# Вычисляем матрицу манхэттенских расстояний для новых локаций
new_distances_matrix = np.zeros((len(new_locations), len(df)))
for i in range(len(new_locations)):
    for j in range(len(df)):
        new_distances_matrix[i, j] = manhattan_distance(new_locations.loc[i, 'x'], new_locations.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])

# Добавляем расстояния до всех локаций в качестве признаков
new_distances_df = pd.DataFrame(new_distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
new_locations = pd.concat([new_locations, new_distances_df], axis=1)

# Предсказания с использованием RandomForest с оптимизированными параметрами
new_locations_pred_rf = best_model_rf.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием RandomForest с оптимизированными параметрами: {new_locations_pred_rf}")

# Предсказания с использованием CatBoost с оптимизированными параметрами
new_locations_pred_catboost = best_model_catboost.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием CatBoost с оптимизированными параметрами: {new_locations_pred_catboost}")

# Предсказания с использованием XGBoost с оптимизированными параметрами
new_locations_pred_xgboost = best_model_xgboost.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием XGBoost с оптимизированными параметрами: {new_locations_pred_xgboost}")

# Предсказания с использованием LightGBM с оптимизированными параметрами
new_locations_pred_lightgbm = best_model_lightgbm.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием LightGBM с оптимизированными параметрами: {new_locations_pred_lightgbm}")